In [ ]:
import cv2
import os
import numpy as np
from skimage.measure import label, regionprops
from skimage.segmentation import watershed
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from skimage.morphology import skeletonize
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025")
import ani


In [ ]:
def estimate_binary_height(binary, th_low, th_high):
    labeled = label(binary)
    heights = [p.bbox[2] - p.bbox[0]
               for p in regionprops(labeled)
               if th_low <= (p.bbox[2] - p.bbox[0]) <= th_high]
    if not heights:
        return None
    mu, sigma = np.mean(heights), np.std(heights)
    return int(mu - 0.5 * sigma), int(mu + 0.5 * sigma)

def load_and_preprocess_image(image_path):
    gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, bin_img = cv2.threshold(gray, 0, 255,
                              cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilated = cv2.dilate(bin_img, kernel, iterations=1)
    return np.bitwise_not(dilated ),dilated

def random_label_cmap(num_labels, seed=25):
    np.random.seed(seed)
    colors = np.random.rand(num_labels, 3)
    colors = np.vstack([[0, 0, 0], colors])
    return mcolors.ListedColormap(colors)

def filter_lines(gray, scales, eta):
    responses = []
    for scale in scales:
        raw = ani.anigauss(
            gray.astype(np.float64), sigv=scale, sigu=eta * scale, phi=0, derv=2, deru=0)
        responses.append(raw.flatten())
    max_resp = np.max(np.array(responses), axis=0).reshape(gray.shape)
    thr = np.mean(max_resp) + 0.4 * np.std(max_resp)
    return (max_resp > thr).astype(np.uint8) * 255

def compute_up_down_distances(binary_image, d):

    binary = (binary_image == 0).astype(np.uint8)
    h, w = binary.shape

    up_dist = np.full((h, w), d, dtype=np.uint16)
    down_dist = np.full((h, w), d, dtype=np.uint16)

    for i in range(1, d + 1):
        shifted = np.zeros_like(binary)
        shifted[i:, :] = binary[:-i, :]
        mask = (binary == 0) & (shifted == 1)
        up_dist[mask] = np.minimum(up_dist[mask], i)
    for i in range(1, d + 1):
        shifted = np.zeros_like(binary)
        shifted[:-i, :] = binary[i:, :]
        mask = (binary == 0) & (shifted == 1)
        down_dist[mask] = np.minimum(down_dist[mask], i)

    total_vertical_distance = up_dist + down_dist
    binary_mask = np.where((binary_image == 255) & (total_vertical_distance>d), 255, 0).astype(np.uint8)
    return binary_mask

In [ ]:
folder_mappings = {
    "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/img-Latin2/test": (
        "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/text-line-prediction-Latin2/test",
        "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/text-line-prediction-Latin2/colored_test"
    ),
    "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/img-Latin14396/test": (
        "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/text-line-prediction-Latin14396/test",
        "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/text-line-prediction-Latin14396/colored_test"
    ),
    "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/img-Syr341/test": (
        "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/text-line-prediction-Syr341/test",
        "/content/drive/MyDrive/Colab Notebooks/icdar_fest_14_02_2025/U-DIADS-TL-test/text-line-prediction-Syr341/colored_test"
    ),
}

for input_folder, (binary_output_root, color_output_root) in folder_mappings.items():
    for root, _, files in os.walk(input_folder):
        for file in files:
            if not file.lower().endswith(".jpg"):
                continue

            print(f"Processing: {file}")
            image_path = os.path.join(root, file)

            # Construct mirrored output path
            relative_path = os.path.relpath(root, start=input_folder)
            binary_out_dir = os.path.join(binary_output_root, relative_path)
            color_out_dir = os.path.join(color_output_root, relative_path)
            os.makedirs(binary_out_dir, exist_ok=True)
            os.makedirs(color_out_dir, exist_ok=True)

            out_path_binary = os.path.join(binary_out_dir, os.path.splitext(file)[0] + ".png")
            out_path_color = os.path.join(color_out_dir, os.path.splitext(file)[0] + ".png")

            # 1) Load and preprocess
            bin, inv_bin = load_and_preprocess_image(image_path)

            # 2) Estimate character height range
            th_low = bin.shape[0] * 0.001
            th_high = bin.shape[0] * 0.1
            height_range = estimate_binary_height(inv_bin, th_low, th_high)

            # 3) Generate line masks
            scales = [height_range[0], height_range[1]]
            lines_mask = filter_lines(bin, scales, eta=2)
            v_separators = compute_up_down_distances(255-lines_mask, 10 * height_range[0])
            inverted_v_separators = cv2.bitwise_not(v_separators)
            lines_mask_split = cv2.bitwise_and(lines_mask, inverted_v_separators)

            # 4) Detect center of lines
            bw = lines_mask_split > 0
            lines_mask_skel = skeletonize(bw).astype(np.uint8) * 255
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 5))
            lines_mask_thick = cv2.dilate(lines_mask_skel, kernel, iterations=1)

            # Filter short blob lines
            num_lab, labels, stats, _ = cv2.connectedComponentsWithStats(lines_mask_thick, connectivity=8)
            lines = np.zeros_like(lines_mask_skel)
            for lbl in range(1, num_lab):
                width = stats[lbl, cv2.CC_STAT_WIDTH]
                height = stats[lbl, cv2.CC_STAT_HEIGHT]
                if width >= 20*height_range[0] and height <= 10*height_range[0]:
                    lines[labels == lbl] = 255

            # 5) Dilate lines region
            d = 2 * height_range[0]
            kern_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*d+1, 2*d+1))
            dilated = cv2.dilate(lines.astype(np.uint8), kern_ellipse)

            # 6) Final intersection with binary image
            final_mask = (((dilated > 0) & (inv_bin > 0)) * 255).astype(np.uint8)

            # 7) Watershed segmentation
            lines_bool = lines > 0
            markers = label(lines_bool)
            distance = ndi.distance_transform_edt(lines_bool)
            final_mask_bool = final_mask > 0
            combined = lines_bool | final_mask_bool
            segmented_lines = watershed(image=distance, markers=markers, mask=combined, watershed_line=True, connectivity=2)

            # Use a random label colormap
            num_labels = int(segmented_lines.max()) + 1
            cmap = random_label_cmap(num_labels, seed=25)

            # Save color output
            plt.imsave(out_path_color, segmented_lines, cmap=cmap)

            # Save binary output
            cv2.imwrite(out_path_binary, (segmented_lines > 0).astype(np.uint8) * 255)


Processing: 251.jpg
Processing: 076.jpg
Processing: 079.jpg
Processing: 277.jpg
Processing: 229.jpg
Processing: 203.jpg
Processing: 252.jpg
Processing: 275.jpg
Processing: 095.jpg
Processing: 200.jpg
Processing: 128.jpg
Processing: 159.jpg
Processing: 117.jpg
Processing: 230.jpg
Processing: 115.jpg
Processing: 108.jpg
Processing: 136.jpg
Processing: 276.jpg
Processing: 038.jpg
Processing: 264.jpg
Processing: 085.jpg
Processing: 032.jpg
Processing: 251.jpg
Processing: 034.jpg
Processing: 253.jpg
Processing: 036.jpg
Processing: 223.jpg
Processing: 047.jpg
Processing: 060.jpg
Processing: 270.jpg
Processing: 071.jpg
Processing: 150.jpg
Processing: 167.jpg
Processing: 075.jpg
Processing: 053.jpg
Processing: 073.jpg
Processing: 031.jpg
Processing: 290.jpg
Processing: 252.jpg
Processing: 190.jpg
Processing: 224.jpg
Processing: 286.jpg
Processing: 313.jpg
Processing: 362.jpg
Processing: 368.jpg
